## 第3章　集約

In [1]:
import numpy as np
import pandas as pd

In [2]:
reserve_tb = pd.read_csv('reserve.csv')

print(reserve_tb.dtypes)
print(reserve_tb.shape)
reserve_tb.head()

reserve_id          object
hotel_id            object
customer_id         object
reserve_datetime    object
checkin_date        object
checkin_time        object
checkout_date       object
people_num           int64
total_price          int64
dtype: object
(4030, 9)


,reserve_id,hotel_id,customer_id,reserve_datetime,checkin_date,checkin_time,checkout_date,people_num,total_price
0,r1,h_75,c_1,2016-03-06 13:09:42,2016-03-26,10:00:00,2016-03-29,4,97200
1,r2,h_219,c_1,2016-07-16 23:39:55,2016-07-20,11:30:00,2016-07-21,2,20600
2,r3,h_179,c_1,2016-09-24 10:03:17,2016-10-19,09:00:00,2016-10-22,2,33600
3,r4,h_214,c_1,2017-03-08 03:20:10,2017-03-29,11:00:00,2017-03-30,4,194400
4,r5,h_16,c_1,2017-09-05 19:50:37,2017-09-22,10:30:00,2017-09-23,3,68100


### 3-1　データ数、種類数の算出

In [3]:
rsv_cnt_tb = reserve_tb.groupby('hotel_id').size()
rsv_cnt_tb.head()

hotel_id
h_1      10
h_10      3
h_100    20
h_101    17
h_102    13
dtype: int64

In [4]:
rsv_cnt_tb = reserve_tb.groupby('hotel_id').size().reset_index()
rsv_cnt_tb.head()

,hotel_id,0
0,h_1,10
1,h_10,3
2,h_100,20
3,h_101,17
4,h_102,13


In [5]:
rsv_cnt_tb.columns = ['hotel_id', 'reserve_cnt']
rsv_cnt_tb.head()

,hotel_id,reserve_cnt
0,h_1,10
1,h_10,3
2,h_100,20
3,h_101,17
4,h_102,13


In [6]:
cus_cnt_tb = reserve_tb.groupby('hotel_id')['customer_id'].nunique()
cus_cnt_tb.head()

hotel_id
h_1      10
h_10      3
h_100    19
h_101    17
h_102    13
Name: customer_id, dtype: int64

In [7]:
cus_cnt_tb = reserve_tb.groupby('hotel_id')['customer_id'].nunique().reset_index()
cus_cnt_tb.head()

,hotel_id,customer_id
0,h_1,10
1,h_10,3
2,h_100,19
3,h_101,17
4,h_102,13


In [8]:
cus_cnt_tb.columns = ['hotel_id', 'cus_cnt']
cus_cnt_tb.head()

,hotel_id,cus_cnt
0,h_1,10
1,h_10,3
2,h_100,19
3,h_101,17
4,h_102,13


In [9]:
pd.merge(rsv_cnt_tb, cus_cnt_tb, on='hotel_id').head()

,hotel_id,reserve_cnt,cus_cnt
0,h_1,10,10
1,h_10,3,3
2,h_100,20,19
3,h_101,17,17
4,h_102,13,13


In [10]:
result = reserve_tb.groupby('hotel_id').agg({'reserve_id': 'count', 'customer_id': 'nunique'})
result.head()

,reserve_id,customer_id
hotel_id,,
h_1,10,10
h_10,3,3
h_100,20,19
h_101,17,17
h_102,13,13


In [11]:
result.reset_index(inplace=True)
result.columns = ['hotel_id', 'rsv_cnt', 'cus_cnt']

result.head()

,hotel_id,rsv_cnt,cus_cnt
0,h_1,10,10
1,h_10,3,3
2,h_100,20,19
3,h_101,17,17
4,h_102,13,13


### 3-2　合計値の算出

In [12]:
reserve_tb = pd.read_csv('reserve.csv')

print(reserve_tb.shape)
reserve_tb.head()

(4030, 9)


,reserve_id,hotel_id,customer_id,reserve_datetime,checkin_date,checkin_time,checkout_date,people_num,total_price
0,r1,h_75,c_1,2016-03-06 13:09:42,2016-03-26,10:00:00,2016-03-29,4,97200
1,r2,h_219,c_1,2016-07-16 23:39:55,2016-07-20,11:30:00,2016-07-21,2,20600
2,r3,h_179,c_1,2016-09-24 10:03:17,2016-10-19,09:00:00,2016-10-22,2,33600
3,r4,h_214,c_1,2017-03-08 03:20:10,2017-03-29,11:00:00,2017-03-30,4,194400
4,r5,h_16,c_1,2017-09-05 19:50:37,2017-09-22,10:30:00,2017-09-23,3,68100


In [13]:
result = reserve_tb.groupby(['hotel_id', 'people_num'])['total_price'].sum()
result.head()

hotel_id  people_num
h_1       1             156600
          2             156600
          3             391500
          4             417600
h_10      1              11200
Name: total_price, dtype: int64

In [14]:
result = reserve_tb.groupby(['hotel_id', 'people_num'])['total_price'].sum().reset_index()
result.head()

,hotel_id,people_num,total_price
0,h_1,1,156600
1,h_1,2,156600
2,h_1,3,391500
3,h_1,4,417600
4,h_10,1,11200


In [15]:
result.rename(columns={'total_price': 'price_sum'}, inplace=True)
result.head()

,hotel_id,people_num,price_sum
0,h_1,1,156600
1,h_1,2,156600
2,h_1,3,391500
3,h_1,4,417600
4,h_10,1,11200


### 3-3　極値、代表値の算出

In [16]:
result = reserve_tb.groupby('hotel_id').agg({'total_price': ['max', 'min', 'mean', 'median', lambda x: np.percentile(x, q=20)]})
result.head()

total_price                                         
                 max    min           mean  median <lambda_0>
hotel_id                                                     
h_1           208800  26100  112230.000000  104400      73080
h_10           67200  11200   42933.333333   50400      26880
h_100          57600   4800   27600.000000   28800       9600
h_101         168000  14000   75764.705882   56000      30800
h_102          72000  12000   32769.230769   24000      18000

In [17]:
result = reserve_tb.groupby('hotel_id').agg({'total_price': ['max', 'min', 'mean', 'median', lambda x: np.percentile(x, q=20)]}).reset_index()
result.head()

hotel_id total_price                                         
                   max    min           mean  median <lambda_0>
0      h_1      208800  26100  112230.000000  104400      73080
1     h_10       67200  11200   42933.333333   50400      26880
2    h_100       57600   4800   27600.000000   28800       9600
3    h_101      168000  14000   75764.705882   56000      30800
4    h_102       72000  12000   32769.230769   24000      18000

In [18]:
result.columns = ['hotel_id', 'price_max', 'price_min', 'price_mean', 'price_median', 'price_20per']
result.head()

,hotel_id,price_max,price_min,price_mean,price_median,price_20per
0,h_1,208800,26100,112230.000000,104400,73080
1,h_10,67200,11200,42933.333333,50400,26880
2,h_100,57600,4800,27600.000000,28800,9600
3,h_101,168000,14000,75764.705882,56000,30800
4,h_102,72000,12000,32769.230769,24000,18000


### 3-4　ばらつき具合の算出

In [19]:
# var:：不偏分散（標本分散）、std：不偏標準偏差（標本標準偏差）→両方とも分母がn-1
result = reserve_tb.groupby('hotel_id').agg({'total_price': ['var', 'std']})
result.head()

total_price              
                   var           std
hotel_id                            
h_1       3.186549e+09  56449.526127
h_10      8.258133e+08  28736.968061
h_100     3.198316e+08  17883.835689
h_101     2.402441e+09  49014.703676
h_102     3.576923e+08  18912.755159

In [20]:
result = reserve_tb.groupby('hotel_id').agg({'total_price': ['var', 'std']}).reset_index()
result.head()

hotel_id   total_price              
                     var           std
0      h_1  3.186549e+09  56449.526127
1     h_10  8.258133e+08  28736.968061
2    h_100  3.198316e+08  17883.835689
3    h_101  2.402441e+09  49014.703676
4    h_102  3.576923e+08  18912.755159

In [21]:
result.columns = ['hotel_id', 'price_var', 'price_std']
result.head()

,hotel_id,price_var,price_std
0,h_1,3.186549e+09,56449.526127
1,h_10,8.258133e+08,28736.968061
2,h_100,3.198316e+08,17883.835689
3,h_101,2.402441e+09,49014.703676
4,h_102,3.576923e+08,18912.755159


In [22]:
result.isnull().sum()

hotel_id     0
price_var    0
price_std    0
dtype: int64

In [23]:
result.fillna(0, inplace=True)
result.head()

,hotel_id,price_var,price_std
0,h_1,3.186549e+09,56449.526127
1,h_10,8.258133e+08,28736.968061
2,h_100,3.198316e+08,17883.835689
3,h_101,2.402441e+09,49014.703676
4,h_102,3.576923e+08,18912.755159


In [24]:
# 列名指定も可
result.fillna(value={'price_var':0, 'price_std':0}, inplace=True)
result.head()

,hotel_id,price_var,price_std
0,h_1,3.186549e+09,56449.526127
1,h_10,8.258133e+08,28736.968061
2,h_100,3.198316e+08,17883.835689
3,h_101,2.402441e+09,49014.703676
4,h_102,3.576923e+08,18912.755159


### 3-5　最頻値の算出

In [25]:
# roundは四捨五入
reserve_tb['total_price'].round(-3).mode()

0    10000
1    20000
2    40000
dtype: int64

### 3-6　順位の算出

In [26]:
reserve_tb = pd.read_csv('reserve.csv')
print(reserve_tb.shape)
reserve_tb.head()

(4030, 9)


,reserve_id,hotel_id,customer_id,reserve_datetime,checkin_date,checkin_time,checkout_date,people_num,total_price
0,r1,h_75,c_1,2016-03-06 13:09:42,2016-03-26,10:00:00,2016-03-29,4,97200
1,r2,h_219,c_1,2016-07-16 23:39:55,2016-07-20,11:30:00,2016-07-21,2,20600
2,r3,h_179,c_1,2016-09-24 10:03:17,2016-10-19,09:00:00,2016-10-22,2,33600
3,r4,h_214,c_1,2017-03-08 03:20:10,2017-03-29,11:00:00,2017-03-30,4,194400
4,r5,h_16,c_1,2017-09-05 19:50:37,2017-09-22,10:30:00,2017-09-23,3,68100


In [27]:
reserve_tb['reserve_datetime'] = pd.to_datetime(
reserve_tb['reserve_datetime'], format='%Y-%m-%d %H:%M:%S')

reserve_tb.head()

,reserve_id,hotel_id,customer_id,reserve_datetime,checkin_date,checkin_time,checkout_date,people_num,total_price
0,r1,h_75,c_1,2016-03-06 13:09:42,2016-03-26,10:00:00,2016-03-29,4,97200
1,r2,h_219,c_1,2016-07-16 23:39:55,2016-07-20,11:30:00,2016-07-21,2,20600
2,r3,h_179,c_1,2016-09-24 10:03:17,2016-10-19,09:00:00,2016-10-22,2,33600
3,r4,h_214,c_1,2017-03-08 03:20:10,2017-03-29,11:00:00,2017-03-30,4,194400
4,r5,h_16,c_1,2017-09-05 19:50:37,2017-09-22,10:30:00,2017-09-23,3,68100


In [28]:
reserve_tb['log_no'] = reserve_tb \
.groupby('customer_id')['reserve_datetime'] \
.rank(ascending=True, method='first')

reserve_tb.head()

,reserve_id,hotel_id,customer_id,reserve_datetime,checkin_date,checkin_time,checkout_date,people_num,total_price,log_no
0,r1,h_75,c_1,2016-03-06 13:09:42,2016-03-26,10:00:00,2016-03-29,4,97200,1.0
1,r2,h_219,c_1,2016-07-16 23:39:55,2016-07-20,11:30:00,2016-07-21,2,20600,2.0
2,r3,h_179,c_1,2016-09-24 10:03:17,2016-10-19,09:00:00,2016-10-22,2,33600,3.0
3,r4,h_214,c_1,2017-03-08 03:20:10,2017-03-29,11:00:00,2017-03-30,4,194400,4.0
4,r5,h_16,c_1,2017-09-05 19:50:37,2017-09-22,10:30:00,2017-09-23,3,68100,5.0


In [29]:
rsv_cnt_tb = reserve_tb.groupby('hotel_id').size()
rsv_cnt_tb.head()

hotel_id
h_1      10
h_10      3
h_100    20
h_101    17
h_102    13
dtype: int64

In [30]:
rsv_cnt_tb = reserve_tb.groupby('hotel_id').size().reset_index()
rsv_cnt_tb.head()

,hotel_id,0
0,h_1,10
1,h_10,3
2,h_100,20
3,h_101,17
4,h_102,13


In [32]:
rsv_cnt_tb.columns = ['hotel_id', 'rsv_cnt']
rsv_cnt_tb.head()

,hotel_id,rsv_cnt
0,h_1,10
1,h_10,3
2,h_100,20
3,h_101,17
4,h_102,13


In [34]:
rsv_cnt_tb['rsv_cnt_rank'] = rsv_cnt_tb['rsv_cnt'].rank(ascending=False, method='min')
rsv_cnt_tb.head()

,hotel_id,rsv_cnt,rsv_cnt_rank
0,h_1,10,235.0
1,h_10,3,300.0
2,h_100,20,12.0
3,h_101,17,43.0
4,h_102,13,139.0


In [35]:
rsv_cnt_tb.drop('rsv_cnt', axis=1, inplace=True)
rsv_cnt_tb.head()

,hotel_id,rsv_cnt_rank
0,h_1,235.0
1,h_10,300.0
2,h_100,12.0
3,h_101,43.0
4,h_102,139.0
